In [1]:
import os
os.environ['HF_HOME'] = os.path.join(os.getcwd(), 'hf_cache')

from transformers import XLNetConfig, XLNetModel, XLNetTokenizer, XLNetLMHeadModel 
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorForPermutationLanguageModeling
from tqdm import tqdm

In [2]:
tokenizer = XLNetTokenizer(vocab_file='models/smiles_sp.model',
                           do_lower_case=False,
                           keep_accents=True
                           )
tokenizer_lc = XLNetTokenizer(vocab_file='models/smiles_sp.model',
                           do_lower_case=True,
                           keep_accents=True
                           )
tokenizer_ka = XLNetTokenizer(vocab_file='models/smiles_sp.model',
                           do_lower_case=False,
                           keep_accents=False
                           )

In [3]:
dataset = load_dataset('csv', data_files=['data/ogb_molhiv/train_hiv.csv'])

Using custom data configuration default-0250d14726bc71f8
Reusing dataset csv (e:\molnlp\mol-prop\hf_cache\datasets\csv\default-0250d14726bc71f8\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)
100%|██████████| 1/1 [00:00<00:00, 64.00it/s]


In [4]:
molecule = dataset['train'][1]['smiles']

In [12]:
molecule
molecule_backup = molecule

In [23]:
molecule = molecule.lower()
molecule = molecule_backup

In [24]:
print(molecule)
print(tokenizer(molecule)['input_ids'])
print(tokenizer_lc(molecule)['input_ids'])
print(tokenizer_ka(molecule)['input_ids'])

O=S(=O)(O)CCS(=O)(=O)O
[27, 23, 28, 10, 5, 4, 11, 5, 4, 56, 10, 5, 4, 10, 5, 4, 5, 0, 0]
[67, 74, 23, 88, 10, 74, 4, 11, 74, 4, 339, 10, 74, 4, 10, 74, 4, 74, 0, 0]
[27, 23, 28, 10, 5, 4, 11, 5, 4, 56, 10, 5, 4, 10, 5, 4, 5, 0, 0]


In [54]:
tk = tokenizer('SCCNS%^SC')['input_ids']
print(tk)
print(tokenizer.convert_ids_to_tokens(tk))


[67, 28, 160, 0, 28, 3, 0, 0]
['▁', 'S', 'CCNS', '<unk>', 'S', 'C', '<unk>', '<unk>']


In [38]:
tokenizer.unk_token_id

0

In [59]:
unk_tok = tokenizer.unk_token_id
count = 0
for sample in dataset['train']:
    enc = tokenizer(sample['smiles'])
    found_unk = False
    for tok in enc['input_ids']:
        if tok == unk_tok:
            if not found_unk:
                found_unk = True
        if found_unk and tok != unk_tok:
            count += 1
            break
print(count)

8133


In [61]:
tokenizer.mask_token_id

0